# Recopilación de tweets con tweepy

En este cuaderno aprenderemos cómo usar la API de Twitter para descargarnos tweets según los criterios de búsqueda que nosotros mismo definamos y guardar los datos que necesitamos en un archivo *csv*.   

Twitter tiene varias APIs, y nosotros nos vamos a conectar a la [Search API](https://developer.twitter.com/en/docs/tweets/search/overview) que permite recabar tweets publicados en los últimos siete días. Si quisieramos ampliar la búsqueda a un periodo más largo, tendríamos que recurrir a una versión Premium de esa misma API o algún otro servicio de pago. También podríamos hacerlo con un scraper que no utulice la API de Twitter, como puede ser [este](https://github.com/taspinar/twitterscraper).   

Python tiene una librería llamada **tweepy** que permite acceder a las APIs de Twitter de una forma muy cómoda. Por supuesto que vamos a darle uso.

### 1. Obtener las credenciales.

Si queremos utilizar una API de Twitter, tendremos que registrar una aplicación en el [Application Management](https://apps.twitter.com) de nuestra cuenta.

Ahí se nos pedirá que demos un nombre a nuestra aplicación, pongamos una descripción y apuntemos a una URL (si no tienes tu propia web, pon la URL que quieras). Cuando le des a *Create your Twitter application*, se creará tu aplicación y podrás ver tus dos claves: **Consumer Key** y **Consumer Secret**.

Pero esto no es todo, porque todavía falta obtener los tokens de acceso. Para eso tienes que abrir la pestaña *Key and Access Tokens* y después de darle a *Generate my Access Token and Token Secret* verás tus **Access Token** y **Access Token Secret**.
      
En principio, puedes guardar tus credenciales en este mismo cuaderno, pero imagínate que quieras compartir tu código con más gente, igual no te interesa que se sepan tus claves personales. Para hacer las cosas bien, crea un archivo llamado, por ejemplo, `secret.py`, con el siguiente contenido:

```
CONSUMER_KEY = 'copia tu Consumer Key aquí'
CONSUMER_SECRET = 'copia tu Consumer Secret aquí'
ACCESS_TOKEN = 'copia tu Access Token aquí'
ACCESS_SECRET = 'copia tu Acces Token Secret aquí'
```

### 2. Preparar el esqueleta del programa.

Primero haremos los *imports* necesarios. Además de la librería *tweepy* necesitaremos la de *csv* que nos permitirá crear archivos de este tipo para almacenar nuestros datos. Por último, tenemos que importar todas nuestras claves desde el archivo *secret.py*.

In [ ]:
import csv
import tweepy
from secret import CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET

Y ahora viene lo más interesante.

In [ ]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET) # creamos un objeto de autentificación de la aplicación
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET) # añadimos nuestra autentificación como usuario


api = tweepy.API(auth, wait_on_rate_limit=True,
                 wait_on_rate_limit_notify=True) # creamos un objeto API que hará de interfaz con la API Twitter

# wait_on_rate_limit pone la API en espera cada vez que alcanzamos el límite de llamada

Asi lo tenemos todo preparado para empezar a recopilar tweets.

NOTA: Con la autorización OAuth que hemos utilizado podemos obtener hasta 18000 tweets cada 15 minutos. También podríamos haber autentificado solo la aplicación sin autentificarnos como usuario, sustituyendo la primera línea por `auth = tweepy.AppAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)` y quitando la segunda. Esto nos habría permitido obtenero hasta 45000 tweets cada 15 minutos. Sin embargo, este tipo de autentificación tiene [algunas limitaciones](https://developer.twitter.com/en/docs/basics/authentication/overview/application-only), por eso nos hemos decantado por la autentificación OAuth, aunque ninguna de estas limitaciones nos hubiera impedido hacer lo que nos hemos propuesto en este cuaderno.

### 3. Mi primer llamada a la API

Vamos a empezar por una tarea muy sencilla: encontraremos 20 tweets con el hashtag "#NLP". Para ello utilizaremos el `Cursor` de `tweepy`. Luego imprimiremos los resultados por pantalla. 

In [ ]:
import json

tweets = tweepy.Cursor(api.search, q="#NLP", count=100).items(100)

# api.search: pasamos al Cursor nuestro objeto API con el método que queremos utilizar (search, en nuestro caso)
# q: lo que queremos encontrar en los tweets
# count: cuántos tweets queremos encontrar por request (el máximo es 100)
# .items(un número): número total de tweets que queremos recopilar

for tweet in tweets:
    print(json.dumps(tweet._json, indent=4)) # con un simple print(tweet) habríamos obtenido un resultado menos legible 
    

Parece que la aplicación está funcionando. La API devuelve los tweets en el formato `json` y viendo lo que se ha imprimido, se puede ver qué atributos tiene cada tweet además del propio texto.

Pero no nos damos por satisfechos y a continuación vamos a intentar conseguir lo siguiente:
     
     * decidir qué atributos de cada tweet nos interesan (¿para qué almacenar cosas que no vamos a utilizar?)
     * guardar nuestros tweets en un archivo csv
     * refinar nuestra búsqueda

### 3. Seleccionar los atributos

Vamos a decir que para cada tweet nos gustaría quedarnos con los siguientes datos:

     * id del tweet (en formato string)
     * fecha y hora del tweet
     * texto del tweet
     * id del autor del tweet
     * screen name del autor
     * nombre del autor
     * ubicación del autor (la que pone en su perfil, no es lo mismo que la geolocalización del tweet)
     
Identificaremos los nombres de los atributos gracias al json que hemos imprimido y accederemos a ellos haciendo `tweet.nombre_del_atributo` (`tweet` aquí es el nombre que hemos definido nosotros para nuestro objetos *tweet*, podríamos haber usado cualquier otro nombre).

### 4. Guardar los datos en un csv

Para guardar nuestros tweets, utilizaremos las posibilidades de la librería `csv` que ya hemos importado.   

De momento seguiremos con la buscando tweets con el hashtag #NLP, pero ya podremos seleccionar los atributos que nos interesa guardar y "desechar" el resto.

In [ ]:
with open('ups/tweets.csv', 'a', encoding='utf-8') as f: # abro mi archivo en modo "escritura"
    csvWriter = csv.writer(f) # creamos un objeto de tipo "csv.writer", le pasamos nuestro archivo
    
    # lanzamos una búsqueda e iteramos por los tweets
    for tweet in tweepy.Cursor(api.search, q="#nlp", count=50).items(50):
        # para cada tweet, escribimos una fila en el archivo
        csvWriter.writerow([tweet.id,
                            tweet.text.replace("\n"," "),
                            tweet.user.id,
                            tweet.user.screen_name                       
                           ])
        
# TE TOCA: añade la fecha/hora de creación del tweet, el nombre del autor y su ubicación a la lista de atributos guardados.

# TE TOCA: por la manera de la que hemos abierto el archivo, cada vez que ejecutemos el código el archivo se va
# a crear de nuevo y los datos ya guardados se van a eliminar. Para hacer que la nueva información se añada a la
# que ya existe, solo hay que cambiar una letra en nuestro código. ¿Cómo hacerlo? (pregunta a Google)  

NOTA: Si vas a querer abrir tu csv en Excel, es posible que te encuentres con una serie de problemas:

1. Los saltos de línea ("\n") en el texto del tweet se interpretarán como el comienzo de una nueva fila (ya lo hemos solucionado más arriba eliminando los saltos de línea en `tweet.text`).
2. Los caracteres non-ASCII aparecerán de forma "rara". Para solucionarlo, abre el archivo en un editor de texto plano, copia su contenido y pégalo a Excel.
3. Excel no separará los datos por columnas. Lo puedes solucionar en la pestaña "Data" -> "Text to Columns".

En principio, no tendrás ninguno de estos problemas si abres el archivo en Google Sheets.

### 5. Refinar la búsqueda

En el ejemplo anterior, lo único que le hemos pedido a la API fue que nos trajera tweets con el hashtag #NLP, pero por supuesto podemos ponerle más filtros.  

Una parte de estos filtros se definirá en la propia query (es decir, en el parámetro, "q"). Puedes encontrar una lista de estos operadores [aquí](https://developer.twitter.com/en/docs/tweets/search/guides/standard-operators). Los otros filtros se pasarán como parámetros, los puedes encontrar  [aquí](https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets).   

Ahora vamos a retocar nuestro código para conseguir lo siguiente:

   * buscar tweets con los hashtags #NLP y #python
   * que contengan una imagen o un video
   * que hayan sido tweeteados antes del 11/12/2017 (recuerda que solo se devuelven tweets de los 7 últimos días)

In [ ]:
with open('data/tweets.csv', 'w', encoding='utf-8') as f: # abro mi archivo en modo "escritura"
    csvWriter = csv.writer(f) # creo un objeto de tipo "csv.writer", le paso mi archivo
    
    # lanzo una búsqueda e itero por los tweets
    for tweet in tweepy.Cursor(api.search, q="#nlp #python filter:media", 
                               until="2017-12-11", count=100).items(200):
        # para cada tweet, escribo una fila en el archivo
        csvWriter.writerow([tweet.id,
                            tweet.text,
                            tweet.user.id,
                            tweet.user.screen_name                       
                           ])
        
# TE TOCA: busca tweets 
    # que contengan los hashtags #nlp O #python
    # que no sean retweets
    # que estén escritos en español

### 6. Guardar las coordenadas de los tweets

Otra cosa que nos gustaría hacer es geolocalizar los tweets. Esta información está contenida en los atributos `geo`, `coordinates` y `place`.   

`geo` y `coordinates` contienen exactamente los mismos datos: las coordenadas exactas desde las que se hizo el tweet. La diferencia consiste en que `geo` está semi-deprecado y que en `geo` primero se cita la latitud y luego la longitud, mientras que en `coordinates` es al revés. También si volvemos al json que hemos imprimido, veremos que los dos atributos contienen diccionarios, así que para acceder a las coordenadas tendremos que hacer `tweet.coordinates["coordinates"]. 

La inmensa mayoría de los tweets no van a tener ningún dato en `geo` y `coordinates`, pero algunos sí, así que vamos a incluirlo en nuestro `csv` y a ver si pillamos por lo menos algunos tweets geolocalizados.

En cuanto a `place` este contiene una geolocalización menos precisa del área desde la que se emitió el tweet. Los tweets conteniendo solo información de `place` (sin la de `geo` y `coordinates`) se dan con un poquito más de frecuencia, pero para no complicarnos la vida, no vamos a trabajar con `place` en este tutorial.

In [ ]:
with open('data/tweets.csv', 'w', encoding='utf-8') as f: # abro mi archivo en modo "escritura"
    csvWriter = csv.writer(f) # creamos un objeto de tipo "csv.writer", le pasamos nuestro archivo
    
    # lanzamos una búsqueda e iteramos por los tweets
    for tweet in tweepy.Cursor(api.search, q="#Madrid", 
                               until="2017-12-11", count=100).items(1000):
        # para cada tweet, escribimos una fila en el archivo
        
        if tweet.coordinates:
            lat = tweet.coordinates["coordinates"][1]
            long = tweet.coordinates["coordinates"][0]            
        else:
            long = None
            lat = None
        csvWriter.writerow([tweet.id,
                            tweet.text,
                            tweet.user.id,
                            tweet.user.screen_name,
                            lat,
                            long
                           ])
        

# TE TOCA: cambia el código para que el tweet solo se guarde en el csv si contiene coordenadas.